In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U bitsandbytes # This ensures to install latest version of bitsandbytes
!pip install -q transformers accelerate bitsandbytes huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

In [ ]:
# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the base model with quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Load  LoRA adapter weights directly from your Drive path
adapter_path = "/content/drive/MyDrive/medical_lora_adapter"
model = PeftModel.from_pretrained(base_model, adapter_path)

In [ ]:
# Load tokenizer #this line is the added code
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Function to generate answers
def generate_medical_answer(question):
    # Format the question with the prompt template
    prompt = f"<s>[INST] <<SYS>>\nYou are a helpful, respectful and Medical assistant.\n<</SYS>>\n\nQuestion: answer this question : {question} [/INST]"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        do_sample=True,
        temperature=0.5,
        top_p=1,
        top_k=60,
        eos_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the model's response part (after [/INST])
    response_only = response.split("[/INST]")[-1].strip()
    return response_only

In [ ]:
# Test again with sample question about Panadol
sample_question = " answer this in short: when can I use panadol?"
print("\n Testing with revised prompt for sample question:", sample_question)
sample_answer = generate_medical_answer(sample_question)
sample_answer


 Testing with revised prompt for sample question:  answer this in short: when can I use panadol?


"Answer: When you have pain or fever, it's normal to reach for the medicine cabinet first, but there is no need to rush to your pharmacy. Panadol has been used by millions of people around the world since its introduction over thirty years ago. It offers fast reliable relief from headache, dental pain, period pains and muscular aches and pains. For most types of pain, Panadol works within minutes so that you can get on with what matters - whether thats work, sport or simply relaxing at home. Panadols active ingredients paracetamol (also known as acetaminophen) act directly on nerve endings to reduce pain signals traveling along them to our brains. Paracetamol also reduces body temperature which helps relieve fevers caused by viruses such as colds and flu. Using Panadol does not cure these illnesses, nor will it prevent secondary complications like dehydration. However, using Panadol can help ease symptoms while allowing some patients to recover faster. If youre suffering from severe co

# let test our model

In [ ]:
import json
!pip install nltk
import nltk
from nltk.translate.bleu_score import corpus_bleu

In [ ]:

nltk.download('wordnet') # Download the 'wordnet' resource
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
!pip install nltk
import nltk
from nltk.translate.bleu_score import corpus_bleu
nltk.download('punkt')
nltk.download('wordnet') # Download the 'wordnet' resource
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!pip install rouge
from rouge import Rouge  # Import the Rouge class

In [ ]:
import scipy.spatial
from sentence_transformers import SentenceTransformer

In [ ]:
import scipy.spatial
from sentence_transformers import SentenceTransformer

In [ ]:
# Load the benchmarking questions
with open('/content/benchmarking.json', 'r') as file:
    benchmarking_data = json.load(file)

In [ ]:
results = []

# Iterate over each question, use the model to generate an answer, and save the results
for item in benchmarking_data:
    question = item['question']
    generated_answer = generate_medical_answer(question)
    results.append({
        'question': question,
        'expected_answer': item['answer'],
        'generated_answer': generated_answer
    })

In [ ]:
# Save the results to a new JSON file for later analysis
with open('model_answer_after_fine.json', 'w') as outfile:
  model_answer_after_fine = results
  json.dump(model_answer_after_fine, outfile, indent=4)

print("Answer generation complete. Results saved to 'model_answer_after_fine.json'.")

Answer generation complete. Results saved to 'model_answer_after_fine.json'.


In [ ]:
# Load the data after hyperparameter tuning
with open('/content/model_answer_after_fine.json', 'r') as file:
    data_after_fine = json.load(file)

refs_after = [item['expected_answer'] for item in data_after_fine]
gen_answer = [item['generated_answer'] for item in data_after_fine]


In [ ]:
## BLEU
bleu_refs_after = [[ref.split()] for ref in refs_after]
bleu_cands_after = [cand.split() for cand in gen_answer]
bleu_score_after = corpus_bleu(bleu_refs_after, bleu_cands_after)

In [ ]:
## ROUGE
rouge_after = Rouge()
rouge_scores_after = rouge_after.get_scores(gen_answer, refs_after, avg=True)

## METEOR
meteor_scores_after = [meteor_score([word_tokenize(ref)], word_tokenize(cand)) for ref, cand in zip(refs_after, gen_answer)]
average_meteor_after = sum(meteor_scores_after) / len(meteor_scores_after)






In [ ]:
!pip install bert_score
from bert_score import score as bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
## BERTScore
P_after, R_after, F1_after = bert_score(gen_answer, refs_after, lang="en", verbose=True)
average_bertscore_after = F1_after.mean().item()



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/6 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/3 [00:00<?, ?it/s]

done in 9.78 seconds, 16.97 sentences/sec


In [ ]:
!pip install sacrebleu
import sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.6 MB/s eta 0:00:00


In [ ]:
## chrF
chrf_scores_after = [sacrebleu.metrics.CHRF().corpus_score([cand], [[ref]]) for cand, ref in zip(gen_answer, refs_after)]
average_chrf_after = sum(score.score for score in chrf_scores_after) / len(chrf_scores_after)

In [ ]:

# Load the sentence transformer model
model = SentenceTransformer('all-mpnet-base-v2')


# Function to calculate cosine similarity
def calculate_similarity(reference, candidate):
    emb1 = model.encode(reference)
    emb2 = model.encode(candidate)
    return 1 - scipy.spatial.distance.cosine(emb1, emb2)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
## Semantic Similarity
similarities_after = []
for item in data_after_fine:
    expected_answer = item['expected_answer']
    generated_answer = item['generated_answer']
    similarity = calculate_similarity(expected_answer, generated_answer)
    similarities_after.append(similarity)
average_similarity_after = sum(similarities_after) / len(similarities_after)

In [ ]:
print(f"BLEU Score After: {bleu_score_after}\n")
print("ROUGE Scores After:", rouge_scores_after)
print(f"\nAverage METEOR Score After: {average_meteor_after}\n")
print(f"Average BERTScore After: {average_bertscore_after}\n")
print(f"Average chrF Score After: {average_chrf_after}\n")
print(f"Average Semantic Similarity After: {average_similarity_after:.4f}")


BLEU Score After: 0.005052285330768599

ROUGE Scores After: {'rouge-1': {'r': 0.4067297276558278, 'p': 0.058928665273649455, 'f': 0.10171352477233092}, 'rouge-2': {'r': 0.06690503816963111, 'p': 0.00879651805645533, 'f': 0.015435654447397607}, 'rouge-l': {'r': 0.3620381898692933, 'p': 0.051999073641238826, 'f': 0.0899019749209218}}

Average METEOR Score After: 0.2293492964367081

Average BERTScore After: 0.8295038938522339

Average chrF Score After: 23.80162117226762

Average Semantic Similarity After: 0.6692


| Metric                      | Before hyperparameter Tuning | After hyperparameter Tuning | After Fine-tuning          |
|-----------------------------|-----------------------------|-----------------------------|----------------------------|
| **BLEU Score**              | 0.018365129100319           | 0.021898307634920827        | 0.005052285330768599       |
| **ROUGE-1 Recall**          | 0.2083327370136514          | 0.20629963994949482         | 0.4067297276558278         |
| **ROUGE-1 Precision**       | 0.13953207019063926         | 0.14922652234802114         | 0.058928665273649455       |
| **ROUGE-1 F1-Score**        | 0.16222520573015256         | 0.1678071804988965          | 0.10171352477233092        |
| **ROUGE-2 Recall**          | 0.06774041799676293         | 0.07138029126709163         | 0.06690503816963111        |
| **ROUGE-2 Precision**       | 0.03536968433700631         | 0.04057446320871533         | 0.00879651805645533        |
| **ROUGE-2 F1-Score**        | 0.045267280609500686        | 0.05015519389739996         | 0.015435654447397607       |
| **ROUGE-L Recall**          | 0.19016640263935092         | 0.1904219558088973          | 0.3620381898692933         |
| **ROUGE-L Precision**       | 0.12651104179122089         | 0.13757772836737991         | 0.051999073641238826       |
| **ROUGE-L F1-Score**        | 0.1474447105381115          | 0.15455426517499718         | 0.0899019749209218         |
| **Average METEOR Score**    | 0.1879472613119189          | 0.18235965287193587         | 0.2293492964367081         |
| **Average BERTScore**       | 0.8463773727416992          | 0.8460686802864075          | 0.8295038938522339         |
| **Average chrF Score**      | 29.414598408426983          | 29.417746841236447          | 23.80162117226762          |
| **Avg Semantic Similarity** | 0.5636                      | 0.5466                      | 0.6692                     |



| Metric                      | Before hyperpara Tuning        | After hyperpara Tuning         | Finetuned Model       |
|-----------------------------|----------------------|----------------------|-----------------------|
| BLEU Score                  | 0.018365129100319    | 0.021898307634920827 | 0.0007664290969272603 |
| ROUGE-1 Recall              | 0.2083327370136514   | 0.20629963994949482  | 0.30839290180200474   |
| ROUGE-1 Precision           | 0.13953207019063926  | 0.14922652234802114  | 0.0251627323259803    |
| ROUGE-1 F1-Score            | 0.16222520573015256  | 0.1678071804988965   | 0.04630869296928104   |
| ROUGE-2 Recall              | 0.06774041799676293  | 0.07138029126709163  | 0.030754420516154905  |
| ROUGE-2 Precision           | 0.03536968433700631  | 0.04057446320871533  | 0.0020749606021840165 |
| ROUGE-2 F1-Score            | 0.045267280609500686 | 0.05015519389739996  | 0.0038682895698780783 |
| ROUGE-L Recall              | 0.19016640263935092  | 0.1904219558088973   | 0.283730498052534     |
| ROUGE-L Precision           | 0.12651104179122089  | 0.13757772836737991  | 0.023054777097541865  |
| ROUGE-L F1-Score            | 0.1474447105381115   | 0.15455426517499718  | 0.04243926266267726   |
| Average METEOR Score        | 0.1879472613119189   | 0.18235965287193587  | 0.12741863713256796   |
| Average BERTScore           | 0.8463773727416992   | 0.8460686802864075   | 0.7992855906486511    |
| Average chrF Score          | 29.414598408426983   | 29.417746841236447   | 14.141495675328553    |
| Average Semantic Similarity | 0.5636               | 0.5466               | 0.5194                |
